# Introducción a Pandas

Pandas es una librería de Python utilizada para tratar datos en forma de tabla. Nos permite importar, exportar y hacer las operaciones habituales que nos permiten otras herramientas como Excel o el lenguaje SQL

## Importación y exportación

Podemos importar datos a DataFrames de Pandas de diferentes orígenes y formatos, entre ellos:

* De CSV: con [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)
* De Excel: con [`read_excel`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html)
* De base de datos: con [`read_sql`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html)

Como ejemplo, vamos a importar unos datos que tenemos en CSV con precios de alquileres en los distritos de Madrid:

In [5]:
import pandas as pd

alquiler = pd.read_csv('dat/alquiler-madrid-distritos.csv', index_col=False)
alquiler.head()

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
1,Barajas,2007,2,11.199855
2,Carabanchel,2007,2,11.127661
3,Centro,2007,2,17.746404
4,Chamartín,2007,2,14.386480


También podemos exportar esos datos a CSV haciendo:

In [7]:
alquiler.to_csv('alquiler.csv')

#### Ejercicio 

Comprueba que se ha guardado el fichero correctamente. Antes de nada, tendrás que ubicarlo en tu disco duro. Debería estar en el directorio en el que se guardan los _notebooks_.

#### SQL


In [9]:
import sqlite3
import datetime as dt
import pandas as pd
import numpy as np

In [10]:
conn = sqlite3.connect('pybodb.sqlite') 
# ejemplo con PostgreSQL usando psycopg2 
# import psycopg2 
# conn = psycopg2.connect(database='ejemplodb', user='kiko')  
# ejemplo con MS ACCESS usando pyodbc (sí, en el trabajo tengo que usar estas cosas) 
# import pyodbc 
# conn = pyodbc.connect("Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=ejemplodb;")  
# ...

Vamos a crear un DataFrame que usaremos como una tabla para insertar en la base de datos. Este DataFrame tendrá una columna de fechas, una de medidas de temperatura promedio diaria (inventada), una de precipitación acumulada en 24h (inventada), una columna con el tipo de sensor que midió la temperatura ese día y una última con el sensor que midió la precipitación.

In [11]:
# fechas para cada día del 2014
fechas = pd.date_range(dt.datetime(2014, 1, 1), dt.datetime(2014, 12, 31))
# Temperatura media diaria durante 2014 en algún lugar del hemisferio Norte
tmed = (np.random.randint(-5, 5, size = 365) +
        20 * np.cos(np.arange(0 - 180, 365 - 180) * 2 * np.pi / 365)
        + 10)
# Precipitación acumulada en 24h
prec = (20 * np.abs(np.random.randn(365) *
        np.cos(np.arange(0, 365) * 2 * np.pi / 365)))
# Sensor que midió la temperatura
marcaT = np.random.choice(['marcaT1', 'marcaT2', 'marcaT3'], size = 365)
# Sensor midió la precipitación
marcaP = np.random.choice(['marcaP1', 'marcaP2'], size = 365)
# Creamos el dataframe y lo guardamos en una tabla llamada 'datos'
df = pd.DataFrame(
        np.array([fechas.values, tmed, prec, marcaT, marcaP]).T,
        columns = ['fecha', 'tmedia','precipitacion','sensorT','sensorP'])
df['fecha'] = pd.to_datetime(df['fecha'])
df.to_sql('datos', con = conn, dtype = {'time': 'TIMESTAMP'})

Vamos a crear una segunda tabla para añadir un poco de complejidad a las consultas que hagamos posteriormente a la base de datos. Esta tabla contendrá información de los sensores usados para las medidas.

In [12]:
# fechas para cada día del 2014
sensores = ['marcaT1', 'marcaT2', 'marcaT3',
            'marcaP1', 'marcaP2']
# Precisión de los sensores
precision = [0.1, 0.5, 1,
             2, 5]
df = pd.DataFrame({'sensores': sensores, 'precision': precision})
df.to_sql('sensores', con = conn)

Como SQL no hace distinción de mayúsculas y minúsculas usaré mayúsculas para las palabras clave de SQL y así las distinguiremos un poco mejor. Para hacer una petición se usa SELECT. Veamos como es una consulta para conocer las tablas que existen en la base de datos (esta consulta es específica para SQLite):

In [13]:
# Esto es específico para sqlite
print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn))


       name
0     datos
1  sensores


En la consulta anterior hemos usados varias palabras clave: SELECT, FROM, WHERE. Este tipo de consultas serán de lo más habitual. Vamos a explotar lo que ya sabemos.

In [14]:
df = pd.read_sql("SELECT * FROM sensores;", conn)

In [15]:
print(df)

   index sensores  precision
0      0  marcaT1        0.1
1      1  marcaT2        0.5
2      2  marcaT3        1.0
3      3  marcaP1        2.0
4      4  marcaP2        5.0


Ahora queremos los datos de precipitación de Junio junto con su fecha. Fijaos que el valor superior del rango no es inclusivo (BETWEEN '2014-06-01' AND '2014-07-01' nos da el dato hasta antes de fecha = '2014-07-01'):

In [16]:
df = pd.read_sql("SELECT fecha, precipitacion FROM datos WHERE fecha BETWEEN '2014-06-01' AND '2014-07-01';", conn)

In [17]:
df

,fecha,precipitacion
0,2014-06-01 00:00:00,35.467209
1,2014-06-02 00:00:00,13.411735
2,2014-06-03 00:00:00,23.687941
3,2014-06-04 00:00:00,2.070440
4,2014-06-05 00:00:00,2.548989
5,2014-06-06 00:00:00,11.404002
6,2014-06-07 00:00:00,13.158481
7,2014-06-08 00:00:00,16.277006
8,2014-06-09 00:00:00,21.690216
9,2014-06-10 00:00:00,42.167782


Ahora quiero los datos de temperatura de los sensores con una precisión superior a 0.5 (el sensor marcaT1 es el único que me da precisión superior a 0.5):

In [18]:
df = pd.read_sql(("SELECT fecha, tmedia FROM datos WHERE datos.sensorT = 'marcaT1';"), conn)

In [19]:
df

,fecha,tmedia
0,2014-01-04 00:00:00,-11.910427
1,2014-01-06 00:00:00,-8.833546
2,2014-01-07 00:00:00,-11.786284
3,2014-01-09 00:00:00,-12.674186
4,2014-01-13 00:00:00,-13.380197
...,...,...
107,2014-12-18 00:00:00,-6.609383
108,2014-12-19 00:00:00,-10.674186
109,2014-12-20 00:00:00,-11.733159
110,2014-12-22 00:00:00,-9.833546


Vamos a hacer una consulta que nos dé el mismo resultado pero que enlaza varias consultas (un SELECT dentro de un SELECT). También la represento de forma un poco más estructurada para que sea más legible:

In [26]:
# La misma consulta de antes pero sin saber la precisión de cada uno de los sensores a priori
q = """
SELECT 
    fecha, tmedia 
FROM 
    datos 
WHERE 
    datos.sensorT = 
    (SELECT 
        sensores 
    FROM 
        sensores 
    WHERE 
        precision < 0.5);"""
df = pd.read_sql(q,conn)

In [21]:
df

,fecha,tmedia
0,2014-01-04 00:00:00,-11.910427
1,2014-01-06 00:00:00,-8.833546
2,2014-01-07 00:00:00,-11.786284
3,2014-01-09 00:00:00,-12.674186
4,2014-01-13 00:00:00,-13.380197
...,...,...
107,2014-12-18 00:00:00,-6.609383
108,2014-12-19 00:00:00,-10.674186
109,2014-12-20 00:00:00,-11.733159
110,2014-12-22 00:00:00,-9.833546


Podemos decirle que nos pase solo una serie de valores. Por ejemplo, solo quiero los tres valores más altos de precipitación de diciembre:

In [ ]:
q = """
SELECT 
    fecha, precipitacion 
FROM 
    datos 
WHERE 
    fecha > '2014-11-30'
ORDER BY
    precipitacion DESC
LIMIT
    3"""
df = pd.read_sql(q, conn)

In [ ]:
df

In [ ]:
conn.close() 

#### Ejercicio

Usa la función [`read_excel`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html) para importar a Python alguna tabla de tu interés

## Estructura básica e inspección

Las tablas en `pandas` son objetos de la clase `DataFrames`. Un `DataFrame` consta de dos partes: un índice y los datos propiamente dichos. Las columnas de los datos son de la clase `Series`.

Para consultar las columnas de un `DataFrame`, accedemos a la propiedad `columns`.

In [ ]:
alquiler.columns

Si además queremos saber el tipo del dato, accedemos a la propiedad `dtypes`.

*Nota*: las cadenas de texto se marcan como `object` dentro de un DataFrame

In [ ]:
alquiler.dtypes

Cada `DataFrame` tiene un índice. Si no lo hemos especificado, será un incremental sin relación con nuestros datos. El uso de índices está recomendado cuando tratamos con datos grandes, ya que permite acceder a las filas por _hash_ en lugar de tener que iterar por todas ellas para encontrar el valor que se busca. Los índices también son importantes a la hora de realizar agregaciones y cruces entre tablas.

Para consultar cuál es el índice de un DataFrame, accedemos a la propiedad `index`.

In [ ]:
alquiler.index

Podemos alterarlo con `set_index`. El nuevo índice puede ser una o varias columnas.

In [ ]:
alquiler_nuevo_indice = alquiler.set_index(['distrito', 'ano', 'quarter'])

Una forma rápida de echar un vistazo a los datos es consultas las primeras o últimas filas del DataFrame, con las funciones `head` y `tail`.

In [ ]:
alquiler.head()

In [ ]:
alquiler.tail()

Podemos seleccionar un listado de columnas a devolver de la siguiente forma:

In [ ]:
alquiler[['distrito', 'precio']].head()

Para conocer el número de filas de una tabla hay varias opciones:

In [ ]:
len(alquiler)

In [ ]:
alquiler.shape

##### Nota

El índice no forma parte propiamente de los datos:

In [ ]:
alquiler_nuevo_indice.shape

### Ejercicio

* Carga en un dataframe de pandas el csv `dat/alquiler-madrid-municipios.csv` en una variable que se llame `alquiler_municipios`
* Examina las primeras y últimas filas
* Extrae el número de filas y columnas

## Filtro y selección

Hay tres operadores fundamentales para seleccionar filas y columnas: `loc`, `iloc` y `[]`. La diferencia fundamental entre `loc` e `iloc` es que el primero requiere _etiquetas_ y el segundo, índices numéricos (la `i` inicial viene de `integer`).


### Selección por índices numéricos

Para acceder por posición usando índices numéricos, se usa `iloc[]`, como en los siguientes ejemplos:

In [ ]:
# por defecto, seleccionamos filas
alquiler_nuevo_indice.iloc[200]

In [ ]:
# pero también se pueden seleccionar filas y columnas
# además, usando rangos
alquiler.iloc[3:5, 1:]

In [ ]:
# índices no consecutivos
# recuerda: en python, se empieza a contar en 0
alquiler.iloc[[1, 2, 4], [0, 3]]

In [ ]:
# los índices negativos indican que se empieza a contar desde el final
alquiler.iloc[-3:-1]

### Ejercicio

* Muestra las primeras 5 filas usando `iloc`
* Muestra las últimas 5 filas usando `iloc`

### Selección por etiquetas

Para acceder por _etiquetas_ (es decir, columnas parte del índice), se usa `loc[]`

In [ ]:
alquiler_nuevo_indice.loc[('Centro', 2014, 2)]

In [ ]:
# O un distrito completo
alquiler_nuevo_indice.loc[('Centro')].head()

#### Ejercicio

Muestra sobre `alquiler_nuevo_indice` las filas para distrito `Retiro` y año 2012.

### Selección por condiciones

Para extraer las filas que cumplen una condición, le pasamos al DataFrame una Series de booleanos, o directamente algo que la devuelva.

In [ ]:
alquiler[alquiler.distrito == 'Retiro'].head()

**Nota:** mira cómo en el código anterior hemos seleccionado la columna `distrito` usando un punto, `alquiler.distrito`. Es una alternativa a la notación mediante corchetes, `alquiler['distrito']`.

**Para profundizar:** puedes leer las diferentes ventajas e inconvenientes de la notación punto con respecto a la de corchetes [aquí](https://www.dataschool.io/pandas-dot-notation-vs-brackets/) y decidir cuál prefieres utilizar.

Podemos combinar varias condiciones con `&` (y lógico) y `|` (o lógico)

In [ ]:
# No olvides los paréntesis, es importante por prioridad de operandos!

alquiler[(alquiler.distrito == 'Retiro') & (alquiler.ano == 2012)]

#### Ejercicio

Extrae los nombres de los distritos cuyo precio por metro cuadrado es superior a 15€ en el año y trimestre más reciente del que tenemos datos (míralo imprimiento las últimas filas de la tabla).

In [ ]:
# 1. Imprime las últimas filas de la tabla

In [ ]:
# 2. Haz el filtro. Una vez hecho, saca solo la columna distrito

## Ordenación

Podemos ordenar un DataFrame por una o varias columnas, de forma ascendente o descendente, con [`sort_values`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html)

In [ ]:
alquiler.sort_values('distrito', ascending=True).head()

In [ ]:
alquiler.sort_values(['ano', 'quarter', 'distrito'], ascending=[False, False, True]).head()

#### Ejercicio

Extrae de mayor a menor por precio las filas de la tabla para Tetuán a partir del año 2017.

## Transformación

Nuevas columnas calculadas, cambio de tipo de dato, eliminar una columna



### Crear una columna calculada

Podemos operar sobre las columnas para crear otras nuevas

In [ ]:
# Hago una copia para no modificar el dataframe original
alquiler_2 = alquiler.copy()

alquiler_2['precio_90m'] = alquiler_2.precio * 90
alquiler_2.head()

Las operaciones que no se pueden lanzar directamente sobre la `Series` completa, la ejecutamos por elemento utilizando `apply`

In [ ]:
# Fíjate bien en la función lambda, es una función en una sola línea
alquiler_2['ano_quarter'] = alquiler_2.apply(lambda fila: str(fila.ano) + 'Q' + str(fila.quarter), axis=1)
alquiler_2.head()

#### Ejercicio

Crea una nueva columna `precio_120m` sobre `alquiler_2` que represente el precio de 120 metros cuadrados, pero utilizando `apply` y una función `lambda`.

### Renombrar una columna

Fíjate en la documentación de `rename` y sus ejemplos, [aquí](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html).

Por ejemplo, para renombrar la columna `precio_90m` a `precio_90_metros` sería:

In [ ]:
alquiler_2 = alquiler_2.rename(columns={'precio_90m': 'precio_90_metros'})
alquiler_2.head()

Fíjate en el ejemplo anterior. En general, las funciones de `pandas` crean un nuevo objeto con el resultado de la operación, pero no modifican el actual. En estas funciones, suele haber un parámetro `inplace` con valor por defecto a `False`. Si lo ponemos a `True`, la operación se realiza sobre el objeto que pasamos por parámetro.

#### La importancia de la nomenclatura

Tener buenos nombres de columnas en un DataFrame es importante. Hará mucho más legible nuestro código si nuestras columnas tienen nombres descriptivos, sin caracteres extraños y separados por `_`.

Unos cuantos ejemplos de malos nombres:

* `col1`, `col2`, ..., `colN`: no sabemos qué es cada cosa.
* `precio euros`, `metros cuadrados`: los espacios dificultan escribir código. Por ejemplo, ya no podremos acceder a las columnas con la notación `dataframe.columna`.
* `año`, `variación`, `precio_€`: los caracteres no-asciii (que no son letras no acentuadas ni números) pueden dar problemas al compartir código (p.e. entre Linux y Windows), al exportar / importar, etc. Es mejor evitarlos.
* `PrecioEuros`, `MetrosCuadrados`: aunque es más sutil, el estándar en Python es escribir en snake_case. Es decir, utilizando minúsculas y usando `_` para separar palabras.

Ejemplo de buenos nombres:

* `distrito`, `precio_euros`, `metros_cuadrados`

### Eliminar una columna

Podemos utilizar [`drop`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html).

Por ejemplo:

In [ ]:
alquiler_2 = alquiler_2.drop(columns=['precio_90_metros'])
alquiler_2.head()

#### Cambiar el tipo de dato

Vamos a crear un DataFrame muy simple para verlo.

In [ ]:
prueba = pd.DataFrame({'precio': ['10.50', '15.35', '22.15']})
prueba

In [ ]:
prueba.dtypes

Tenemos un DataFrame con precios, pero estos son cadenas de texto en lugar de números. Esto va a limitar nuestro análisis: no podremos ejecutar operaciones aritméticas, calcular medias, etc.

Es muy habitual que esto pase en el momento de cargar unos datos. Veremos más adelante cómo solucionar problemas al cargar CSVs de habla española, que utilizan la coma como separador decimal en lugar del punto, que es el que entiende Python (y, en general, todos los lenguajes de programación).

Sobre nuestro ejemplo actual `prueba`, podemos usar [`astype`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) para convertir la columna a numérica. En este caso, como es numérica con decimales, usaremos el tipo `float`.

In [ ]:
prueba['precio'] = prueba.precio.astype(float)
prueba.dtypes

## Resumen estadístico

Pandas provee una serie de funciones de resumen estadístico que podemos aplicar sobre una columna concreta, o sobre todas las del DataFrame.

Para un resumen para todas las columnas de número de filas, media, desviación estándar, cuartiles, ... usamos [`describe`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)

In [ ]:
alquiler.describe()

Podemos utilizar también `sum`, `mean`, `std`, `count`, `min`, `max`, ... sobre el DataFrame o una columna en concreto

In [ ]:
alquiler.mean()

In [ ]:
alquiler.precio.max()

#### Ejercicio

Extrae los cuantiles 0.1 y 0.9 del precio para el distrito `Tetuan`.

## Agrupación

De una forma equivalente a como hacemos en SQL, podemos agregar las tablas y sacar resúmenes de los grupos. La operación en pandas se hace en dos fases:

* El `groupby`: donde especificamos la o las columnas por las que agregar
* La aplicación de la función de agregación sobre una o varias columnas

Un resumen usando una función de agregación sobre todas las columnas del DataFrame

In [ ]:
alquiler.groupby('ano').max()

# Atención, fíjate bien en lo que hace esto. Saca el valor máximo de distrito (alfabéticamente),
#  de quarter y precio (numéricamente), pero no representa filas completas
# Es decir, Villaverde en el quarter 4 no tuvo ese precio

Para hacerlo únicamente sobre una columna:

In [ ]:
alquiler.groupby('ano').precio.min()

Para aplicar diferentes resúmenes sobre diferentes columnas

In [ ]:
tmp = alquiler.groupby('ano').agg({'precio': 'mean', 'distrito': 'first'})
tmp.head()

#### Ejercicio

Extrae el precio máximo histórico para cada distrito a partir del 2010

Puedes ver más información sobre agrupaciones en la [documentación de pandas](https://pandas.pydata.org/pandas-docs/stable/groupby.html). Es especialmente útil la parte sobre transformaciones.

## Cruce

Podemos cruzar dos tablas por una o varias columnas en pandas, de forma equivalente a como hacemos en SQL, con [`merge`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html). También podemos usar los distintos tipos de cruce:

* `inner`: para obtener solamente los registros que crucen en ambas tablas
* `left` o `right`: para mantener los registros de una de las dos tablas, crucen o no con la otra
* `outer`: para manter los registros de ambas tablas, crucen o no

In [ ]:
# Aquí, además, un ejemplo de cómo crear un dataframe a partir de un diccionario
df_ejemplo = pd.DataFrame({'distrito':  ['Moratalaz', 'Centro', 'Barajas'],
                           'poblacion': [95000, 150000, 46000]})
df_ejemplo

In [ ]:
tmp = df_ejemplo.merge(alquiler, on='distrito')
tmp.head()

In [ ]:
len(tmp)

In [ ]:
tmp = df_ejemplo.merge(alquiler, on='distrito', how='right')
tmp.tail()

In [ ]:
len(tmp)

#### Ejercicio

* Carga en un DataFrame el CSV `dat/venta-madrid-distritos.csv`
* Crúzalo con el que ya tenemos de alquiler. El objetivo es tener, para cada distrito, año y cuatrimestre, tanto el precio de alquiler como el de venta del metro cuadrado. Para saber cómo poner sufijos a las columnas que colisionan en el cruce, mira la documentación de `merge`.
* Extrae los precios medios de venta y alquiler por distrito para todo el histórico
* Extrae, para el año y cuatrimestre más reciente del que haya datos, el distrito donde es más rentable comprar una vivienda para destinarla a alquiler. Es decir, con el ratio precio alquiler / precio venta más alto.